In [17]:
import pandas as pd
import sys
import os
sys.path.insert(0, '..')
from tqdm import tqdm
import spectral_denoising as sd
from spectral_denoising.noise import *
from spectral_denoising.file_io import read_df, save_df
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
master_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/dilution_series_data'
formula_db = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/formula_db/formulaDB_sorted.csv')
output_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/dilution_series_data/synthetic_data'
data = read_df(os.path.join(master_dir, 'dilution_mapped_unprocessed.csv'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
done read in df...


In [ ]:
from itertools import product
chemical_lamda = 50
electric_lamda = 5
chemcial_abundant = [0.1,0.2,0.5]
electric_abundant = [2,10,100]
combinations = product(chemcial_abundant, electric_abundant)
for combination in (combinations):
    #first always chemical
    msms_conts_all = []
    chemical_ratio = combination[0]
    electri_ratio = combination[1]
    data_current = data.copy()

    for index, row in tqdm(data_current.iterrows(), total = len(data_current)):
        num_peaks = len(row['peaks'])
        noise_chemical = generate_chemical_noise(row['precursor_mz'],chemical_lamda, row['adduct'][-1], formula_db=formula_db, n = np.int64(np.ceil(chemical_ratio*num_peaks)))
        noise_electric = generate_noise(row['precursor_mz'], electric_lamda, n = np.int64(np.ceil(electri_ratio*num_peaks)))
        msms_cont_row = add_noise(row['peaks'], noise_chemical, )
        msms_cont_row = add_noise(msms_cont_row, noise_electric)
        msms_conts_all.append(msms_cont_row)
    data_current['msms_cont']=msms_conts_all
    data_current['entropy_cont_raw'] = [sd.entropy_similairty(data_current['msms_cont'].iloc[i], data_current['library_peaks'].iloc[i], data_current['precursor_mz'].iloc[i]) for i in range(len(data_current))]
    save_df(data_current, os.path.join(output_dir, 'synthetic_noise_sn_'+str(chemical_ratio)+'_'+str(electri_ratio)+'.csv'))